In [6]:
# -*- coding: utf-8 -*-
import pandas as pd
import math
import numpy as np

df_gen1 = pd.read_excel("probe.xls", sheet_name="Gen1", skiprows=3)
df_sxm = pd.read_excel("probe.xls", sheet_name="SXM向け", skiprows=3)
df_vnext = pd.read_excel("probe.xls", sheet_name="vNext", skiprows=1)

#print(df_gen1.columns)
#print(df_sxm.columns)
#print(df_vnext.columns)

df_gen1_flat = pd.read_excel("probe.xls", sheet_name="flat_table_gen1", skiprows=3).fillna({"物理カラム名":"no_label"})
df_sxm_flat = pd.read_excel("probe.xls", sheet_name="flat_table_sxm", skiprows=3).fillna({"物理カラム名":"no_label"})
df_vnext_flat = pd.read_excel("probe.xls", sheet_name="flat_table_vnext", skiprows=3).fillna({"物理カラム名":"no_label"})

#print(df_gen1_flat.columns)
#print(df_sxm_flat.columns)
#print(df_vnext_flat.columns)

#print(df)
lab_val = "0x"
lab_name = ""
lab_size = ""
df_gen1 = df_gen1.fillna({"ラベル値": 'no_lab_val', 'ラベル名': 'no_lab_name', 'tsv項目No.': 'no_no', 'サイズ':'no_size'})
df_sxm = df_sxm.fillna({"ラベル値": 'no_lab_val', 'ラベル名': 'no_lab_name', 'tsv項目No.': 'no_no',  'サイズ':'no_size'})
df_vnext = df_vnext.fillna({"Label_Hex_4digits": 'no_lab_val',  "Name_2": 'no_lab_name', "Size_bytes":'no_size', "Description":'no_des', 'Conversion':'no_size'})

def get_size(in_bytes, size_byte, size_bit):
    by = size_byte
    bt = size_bit
    try:
        if in_bytes.find("byte") != -1:
            #print("find byte:{0}".format(in_bytes[:-4]))
            by += int(in_bytes[:-4])
        else:
            #print("find bit:{0}\n".format(in_bytes[:-3]))
            bt += int(in_bytes[:-3])
    except:
        if type(in_bytes) == str:
            print("{1} {0} get bit error.\n".format(type(in_bytes), in_bytes))
        elif not math.isnan(in_bytes):
            print("{1} {0} get bit error.\n".format(type(in_bytes), in_bytes))
    return by, bt
    
new_row = None
field_num = 0
row_listg1 = []
row_listx = []
row_listvn = []
size_byte = 0
size_bit = 0
for index, row in df_gen1.iterrows():
    if row["ラベル値"] != "no_lab_val":
        if new_row is not None:
            new_row[2] = size_byte + size_bit/8
            row_listg1.append(new_row)
            size_byte = 0
            size_bit = 0
            #print(new_row)
        field_num = 1
        size_byte, size_bit = get_size(row["Byte"], size_byte, size_bit)
        new_row = [row["ラベル値"], row["ラベル名"], size_byte, field_num, 1 if row["サイズ"]=="固定長" else 0]
        #print(index, row["ラベル値"])
    else:
        size_byte, size_bit = get_size(row["Byte"], size_byte, size_bit)
        if row['tsv項目No.'] != 'no_no':
            field_num += 1
            new_row[3] = field_num
        #print(index, row["ラベル値"], row["Byte"])
row_listg1.append(new_row)
new_row = None
field_num = 0
size_byte = 0
size_bit = 0
for index, row in df_sxm.iterrows():
    if row["ラベル値"] != "no_lab_val":
        if new_row is not None:
            new_row[2] = size_byte + size_bit/8
            row_listx.append(new_row)
            size_byte = 0
            size_bit = 0
            #print(new_row)
        field_num = 1
        size_byte, size_bit = get_size(row["Byte"], size_byte, size_bit)
        new_row = [row["ラベル値"], row["ラベル名"], size_byte, field_num, 1 if row["サイズ"]=="固定長" else 0]
        #print(index, row["ラベル値"])
    else:
        size_byte, size_bit = get_size(row["Byte"], size_byte, size_bit)
        if row['tsv項目No.'] != 'no_no':
            field_num += 1
            new_row[3] = field_num
        #print(index, row["ラベル値"], row["Byte"])
row_listx.append(new_row)

new_row = None
label_hex = ""
field_desc = ""
label_size = 0
for index, row in df_vnext.iterrows():
    if row["Label_Hex_4digits"] != "no_lab_val" and row["Label_Hex_4digits"] != label_hex and row["Label_Hex_4digits"] != "00h":
        if new_row is not None:
            row_listvn.append(new_row)
            #print(new_row)
            field_num = 1
        field_desc = row["Description"]
        label_size = int(row["Size_bytes"])
        label_hex = row["Label_Hex_4digits"]
        new_row = [row["Label_Hex_4digits"][:-1], row["Description"], label_size, field_num, 1, field_desc]
        #print(new_row) if (row['Conversion pattern#'] in [31, 32, 33]) else print("pass")
    else:
        if row['Name_2'] != 'Label':
            if row["Label_Hex_4digits"] != "00h":
                field_num += 1
            if row["Size_bytes"] != "no_size":
                label_size += int(row["Size_bytes"])
            des = ""
            if row["Description"] != "no_des":
                des = row["Description"]
            field_desc += "{0}: {1}¥n".format(row["Name_2"], des)
            new_row[2] = label_size
            new_row[3] = field_num
            new_row[5] = field_desc
            if new_row[4] != 0:
                new_row[4] = 0 if (row['Conversion'] in [31, 32, 33]) else 1
                #print(row['Conversion']) if (row['Conversion'] in [31, 32, 33]) else print("pass")
        #print(index, row["ラベル値"], row["Byte"])
row_listvn.append(new_row)
#print(row_listg1)
#print(row_listx)
#print(row_listvn[:][0])
count = 0
field_count = 0
for (index, g1) in enumerate(row_listg1):
    if g1[1] != '未使用':
        count += 1
        field_count += g1[3]
print("Gen1: {0} {1}".format(count, field_count))
count = 0
field_count = 0
for (index, x1) in enumerate(row_listx):
    if x1[1] != '未使用':
        count += 1
        field_count += x1[3]
print("SMX: {0} {1}".format(count, field_count))

field_count = 0
for (index, vn) in enumerate(row_listvn):
    if vn[1] != 'no_lab_name':
        count += 1
        field_count += vn[3]
        #print(vn)
print("VNEXT: {0} {1}".format(count, field_count))

# """
# for g1 in row_listg1:
#     for x in row_listx:
#         if g1[1] == x[1] and g1[1] != '未使用':
#             print(g1)
#             print(x)
# """


1～20byte
 <class 'str'> get bit error.

1～20byte <class 'str'> get bit error.

Gen1: 128 437
SMX: 132 697
VNEXT: 203 1310


In [55]:
# -*- coding: utf-8 -*-
get_size = lambda fields: (fields[0], fields[2]) if fields[2] == 1 else (fields[0]-1, fields[2])
def label_flat(df_flat, label_list, type_name):
    #type_name = bin_file.split("_")[0]
    with open("%s_labels.txt"%type_name, 'w') as flat_f:
        with open("%s_bin"%type_name, 'w') as bin_f:
            with open("%s_dict.txt"%type_name, 'w') as dict_f:
                pre_label = ""
                label_size_flg=[0, 0, 0]
                ret_list = []
                bin_f.write("def mk_{0}data({0}_bin_file):\n".format(type_name))
                dict_f.write("    %s_dict = {\n"%type_name)
                for index, row in df_flat.iterrows():
                    if row["物理カラム名"]  == "no_label":
                        label = pre_label
                        #print(label)
                        continue
                    if len(row["物理カラム名"] .split('_')) > 2:
                        (label, field) = row["物理カラム名"] .split('_')[1:]
                    else:
                        label = row["物理カラム名"] .split('_')[1]
                    if label == "HEADER":
                        pre_label = label
                        flat_f.write('{0}_{1}\n'.format(label, field))
                    elif pre_label != label:
                        pre_label = label
                        try:
                            if len(label) == 5:
                                label_size_flg=list(filter(lambda x:x[0]==label[:-1], label_list))[0][2:5]
                            else:
                                label_size_flg=list(filter(lambda x:x[0]==label, label_list))[0][2:5]
                            #print("{0}:{1}".format(label, label_size))
                        except:
                            label_size_flg=[0,0,0]
                            print("except {0}:{1}".format(label, label_size_flg))
                        if pre_label[-1] == 'h':
                            wr_label = '0x' + pre_label[:-1]
                            flat_f.write('{0} {1} {2}\n'.format(wr_label, label_size_flg[0], label_size_flg[2]))
                            bin_f.write("    {0}_{1} = '>H{2}s' #{3}\n".format(type_name, wr_label, int(label_size_flg[0]), "valuable" if label_size_flg[2] == 0 else "fix"))
                            bin_f.write("    {0}_bin_file.write(struct.pack({0}_{1}, {1}, bytes.fromhex('')))\n".format(type_name, wr_label))
                            dict_f.write('        "{0}":[">{1}s", {2}],\n'.format(wr_label, label_size_flg[0] if label_size_flg[2] ==1 else  label_size_flg[0]-1, label_size_flg[2] ))
                            ret_list.append([wr_label, label_size_flg[0], label_size_flg[2]])
                        else:
                            flat_f.write('{0} {1} {2}\n'.format(pre_label, label_size_flg[0], label_size_flg[2]))
                            bin_f.write("    {0}_{1} = '>H{2}s' #{3}\n".format(type_name, pre_label, int(label_size_flg[0]), "valuable" if label_size_flg[2] == 0 else "fix"))
                            bin_f.write("    {0}_bin_file.write(struct.pack({0}_{1}, {1}, bytes.fromhex('')))\n".format(type_name, pre_label))
                            dict_f.write('        "{0}":[">{1}s", {2}],\n'.format(pre_label, label_size_flg[0] if label_size_flg[2] ==1 else  label_size_flg[0]-1, label_size_flg[2] ))
                            ret_list.append([pre_label, label_size_flg[0], label_size_flg[2]])
                dict_f.write('        }\n')
        return ret_list
    
ret = label_flat(df_gen1_flat, row_listg1, "gen1")
ret = label_flat(df_sxm_flat,  row_listx, "sxm")
ret = label_flat(df_vnext_flat,  row_listvn, "vnext")

except 0001h:[0, 0, 0]


In [11]:
f = lambda x,  y: x if y == 0 else x-1
print(f(1, 0))
print(f(0, 1))

1
-1


In [69]:
print(0x01.bit_length())
get_bits = lambda value: get_bits(value>>1) +  (0x01&(value))  if  value.bit_length() > 0 else 0
get_bits(0x15)

1


3

In [48]:
get_bits = lambda bits, value:  get_bits(bits -1, value) +  (0x01&(value >> bits-1))  if bits > 0 else 0
get_bits(8, 0x70)

3

In [71]:
get_bits = lambda x, y: x + (0x01 & (y >> i)) for i in rang(y.bit_length() )
get_bits(0, 0x1)

SyntaxError: invalid syntax (<ipython-input-71-18c94cfb7556>, line 1)

In [82]:
from functools import reduce
print([0x01>>i for i in range(0x01.bit_length())])
get_bits = lambda bit: reduce(lambda x,y:x+y, [0x1&bit>>i for i in range(bit.bit_length())])
get_bits(0x18)

[1]


2